## Set up datasets and dataloaders

In [1]:
from utils.device import get_device
from utils.data import DatasetConfig
from utils.labels import VINDR_CXR_LABELS, VINDR_SPLIT
from models.backbone.datasets import MEAN_STDS

IMG_PATH = 'datasets/vindr-cxr-png'

NUM_SHOTS = 5
NUM_WAYS = 7
TRAIN_NUM_WAYS= 7
dataset_config = DatasetConfig(IMG_PATH, 'data/vindr_cxr_split_labels.pkl', 'data/vindr_train_query_set.pkl', VINDR_CXR_LABELS, VINDR_SPLIT, MEAN_STDS['chestmnist'])
device =  get_device()

In [ ]:
import torch
import random
import numpy as np

def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    random.seed(seed)

## Models with attention
### Run experiments on proposed model

In [6]:
import torch
from models.attention.model import LabelImageAttention, LabelImagePrototypeModel
from models.embedding.model import ImageTextEmbedding, TextEncoder, ImageEncoder

from utils.prototype import class_prototype_inf
from models.metaclassifier.trainer import ControlledMetaTrainer
from models.metaclassifier.model import ClsModel


encoder = torch.load('models/embedding/model/imgtext_model_trained.pth')
encoder.text_model.device = device
attention = torch.load('models/attention/model/attention-model-8h4l.pth')
model = ClsModel(encoder, attention, 512, class_prototype_inf, fc_hidden_size=16)
mtrainer = ControlledMetaTrainer(model, NUM_SHOTS, NUM_WAYS, dataset_config, train_n_ways=TRAIN_NUM_WAYS, device=device)

In [ ]:
mtrainer.run_eval(mtrainer.model, mtrainer.test_loader)


In [7]:
mtrainer.run_eval(mtrainer.model, mtrainer.val_loader)

(20.361516034985424, 0.5202569099543621, 0.7382618546485901)

In [9]:
# 5 times?
mtrainer.run_train(2, lr=1e-5)

Batch 1: loss 0.6902595162391663 | Acc 53.46938775510204
Batch 2: loss 0.6968538761138916 | Acc 42.44897959183673
Batch 3: loss 0.7003634770711263 | Acc 37.95918367346939
Batch 4: loss 0.6974166184663773 | Acc 53.46938775510204
Batch 5: loss 0.6979597449302674 | Acc 44.08163265306123
Batch 6: loss 0.6973948379357656 | Acc 48.97959183673469
Batch 7: loss 0.6978198204721723 | Acc 44.08163265306123
Batch 8: loss 0.6980315819382668 | Acc 45.714285714285715
Batch 9: loss 0.6969953179359436 | Acc 53.87755102040816
Batch 10: loss 0.6979582905769348 | Acc 39.183673469387756
Batch 11: loss 0.6966771320863203 | Acc 57.55102040816327
Batch 12: loss 0.6973831256230673 | Acc 40.0
Batch 13: loss 0.6974497345777658 | Acc 46.53061224489796
Batch 14: loss 0.6974168930734906 | Acc 46.93877551020408
Batch 15: loss 0.697468622525533 | Acc 44.89795918367347
Batch 16: loss 0.6972164176404476 | Acc 49.38775510204081
Batch 17: loss 0.6974642136517692 | Acc 43.26530612244898
Batch 18: loss 0.6976404885450999 |

In [12]:
mtrainer.run_eval(mtrainer.best_model, mtrainer.test_loader)

(19.93031358885018, 0.507083518313992, 0.7321400235338908)

In [13]:
mtrainer.model.attn_model.set_trainable(True)

In [17]:
mtrainer.run_train(3, lr=1e-5)

Batch 1: loss 0.6971045732498169 | Acc 42.44897959183673
Batch 2: loss 0.6954399049282074 | Acc 48.16326530612245
Batch 3: loss 0.6974401871363322 | Acc 39.183673469387756
Batch 4: loss 0.696072444319725 | Acc 50.61224489795918
Batch 5: loss 0.693632709980011 | Acc 60.0
Batch 6: loss 0.6950256526470184 | Acc 37.55102040816327
Batch 7: loss 0.6943262900624957 | Acc 51.83673469387755
Batch 8: loss 0.6948138996958733 | Acc 42.44897959183673
Batch 9: loss 0.6948209206263224 | Acc 46.93877551020408
Batch 10: loss 0.6951384782791138 | Acc 43.26530612244898
Batch 11: loss 0.6944867806001143 | Acc 52.6530612244898
Batch 12: loss 0.6939454972743988 | Acc 51.42857142857142
Batch 13: loss 0.6941679395162142 | Acc 44.89795918367347
Batch 14: loss 0.6944387938295092 | Acc 46.12244897959184
Batch 15: loss 0.6944648385047912 | Acc 48.16326530612245
Batch 16: loss 0.6943309046328068 | Acc 47.3469387755102
Batch 17: loss 0.6938948526101953 | Acc 55.51020408163265
Batch 18: loss 0.6942294736703237 | Acc

In [18]:
# Model trained 3 epochs with lr=5e-6, 3 epochs with lr=1e-6, 3 epochs with lr=1e-5
mtrainer.run_eval(mtrainer.model, mtrainer.test_loader)

(78.01891488302637, 0.4961423906653175, 0.6495074789698531)

In [19]:
mtrainer.run_eval(mtrainer.best_model, mtrainer.test_loader)

(79.21353907416626, 0.5099220733360005, 0.6617257754977156)

In [20]:
torch.save(mtrainer.best_model.cls.state_dict(), 'models/metaclassifier/model/comb3/cls_weights-16.pkl')

In [21]:
torch.save(mtrainer.best_model.attn_model, 'models/metaclassifier/model/comb3/attention-model-8h4l.pth')

### Prototypical Network with attention

In [2]:
import torch
from models.attention.model import LabelImageAttention, LabelImagePrototypeModel
from models.embedding.model import ImageTextEmbedding, TextEncoder, ImageEncoder

from utils.prototype import class_prototype_inf
from models.metaclassifier.base import euclidean_distance
from models.metaclassifier.trainer import ControlledMetaTrainer
from models.metaclassifier.model import ProtoNetAttention

device =  get_device()

encoder = torch.load('models/embedding/model/imgtext_model_trained.pth')
encoder.text_model.device = device
attention = torch.load('models/attention/model/attention-model-8h4l.pth')
# imgtxt_encoder, attn_model, class_prototype_aggregator, distance_func
model = ProtoNetAttention(encoder, attention, class_prototype_inf, euclidean_distance)
mtrainer = ControlledMetaTrainer(model, NUM_SHOTS, NUM_WAYS, dataset_config, device=device)

In [3]:
mtrainer.run_eval(model, mtrainer.test_loader, True)

/Users/naomileow/Documents/school/CS6240/project/utils/prototype.py:55: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  classes_count = torch.nonzero(label_inds)[:,1].bincount()


Loss 0.6663146018981934 | Accuracy 81.63265306122449 | AUC 0.5520890348476556
Loss 0.6675408482551575 | Accuracy 77.9591836734694 | AUC 0.592805177243311
Loss 0.6662786602973938 | Accuracy 80.81632653061224 | AUC 0.5031611300576818
Loss 0.6649683117866516 | Accuracy 80.0 | AUC 0.5192899818312419
Loss 0.6666518449783325 | Accuracy 80.0 | AUC 0.5661068620956023
Loss 0.665653645992279 | Accuracy 80.81632653061224 | AUC 0.5215153647665963
Loss 0.6630535125732422 | Accuracy 81.63265306122449 | AUC 0.48086569424007847
Loss 0.6627781987190247 | Accuracy 81.63265306122449 | AUC 0.5596055745021262
Loss 0.6620592474937439 | Accuracy 82.44897959183673 | AUC 0.45533174308684515
Loss 0.6656913161277771 | Accuracy 79.59183673469387 | AUC 0.5300442635824945
Loss 0.666347324848175 | Accuracy 81.22448979591836 | AUC 0.5316662740132128
Loss 0.6675103306770325 | Accuracy 79.18367346938776 | AUC 0.5914888911229517
Loss 0.6687719821929932 | Accuracy 77.9591836734694 | AUC 0.5190180989547633
Loss 0.66781353

In [4]:
mtrainer.run_eval(model, mtrainer.val_loader)

(79.37026239067053, 0.5161583489962621, 0.6703500832830157)

In [5]:
mtrainer.run_train(2, lr=1e-4)

Batch 1: loss 0.645804762840271 | Acc 59.795918367346935
Batch 2: loss 0.64562126994133 | Acc 58.877551020408156
Batch 3: loss 0.6446925004323324 | Acc 60.51020408163266
Batch 4: loss 0.6453973799943924 | Acc 59.285714285714285
Batch 5: loss 0.6450785756111145 | Acc 58.673469387755105
Batch 6: loss 0.6450744767983755 | Acc 59.38775510204082
Batch 7: loss 0.6451364670481 | Acc 58.57142857142858
Batch 8: loss 0.6450803205370903 | Acc 60.71428571428571
Batch 9: loss 0.6445973383055793 | Acc 61.530612244897966
Batch 10: loss 0.6443249225616455 | Acc 60.71428571428571
Batch 11: loss 0.6443740454587069 | Acc 59.591836734693885
Batch 12: loss 0.6442251602808634 | Acc 60.10204081632653
Batch 13: loss 0.6441494272305415 | Acc 60.0
Batch 14: loss 0.6442059363637652 | Acc 57.44897959183673
Batch 15: loss 0.6439297715822856 | Acc 62.755102040816325
Batch 16: loss 0.6437373459339142 | Acc 59.08163265306122
Batch 17: loss 0.6442396991393146 | Acc 56.93877551020409
Batch 18: loss 0.644419295920266 | 

In [7]:
print(mtrainer.best_model.scale)

Parameter containing:
tensor(1.0008, device='mps:0', requires_grad=True)


In [8]:
mtrainer.run_eval(mtrainer.best_model, mtrainer.test_loader)

(80.38825286212047, 0.5417728137567698, 0.6569410795118751)

## Run experiments on baseline models without attention
### RelationNet

In [32]:
import torch
from models.attention.model import LabelImageAttention, LabelImagePrototypeModel
from models.embedding.model import ImageTextEmbedding, TextEncoder, ImageEncoder

from utils.prototype import class_prototype_inf
from models.metaclassifier.trainer import ControlledMetaTrainer
from models.metaclassifier.baselines import RelationNet

encoder = torch.load('models/embedding/model/imgtext_model_trained.pth')
encoder.text_model.device = device
base_model = RelationNet(encoder, 512, class_prototype_inf, fc_hidden_size=16)
btrainer = ControlledMetaTrainer(base_model, NUM_SHOTS, NUM_WAYS, dataset_config, device=device)

In [33]:
btrainer.run_eval(btrainer.model, btrainer.val_loader)

(21.01457725947522, 0.5016376709062274, 0.6972464016505651)

In [34]:
btrainer.run_eval(btrainer.model, btrainer.test_loader)

(20.079641612742655, 0.5146355623202179, 0.6978601042817278)

In [43]:
btrainer.run_train(2, lr=1e-6)

Batch 1: loss 0.679679274559021 | Acc 68.16326530612244
Batch 2: loss 0.6872178614139557 | Acc 48.16326530612245
Batch 3: loss 0.6854563554128011 | Acc 63.6734693877551
Batch 4: loss 0.6872857213020325 | Acc 53.46938775510204
Batch 5: loss 0.6890603423118591 | Acc 52.6530612244898
Batch 6: loss 0.687427838643392 | Acc 66.12244897959184
Batch 7: loss 0.6865879552704948 | Acc 62.04081632653061
Batch 8: loss 0.6865909770131111 | Acc 61.224489795918366
Batch 9: loss 0.6880093084441291 | Acc 43.673469387755105
Batch 10: loss 0.6870020091533661 | Acc 68.57142857142857
Batch 11: loss 0.6873770952224731 | Acc 53.87755102040816
Batch 12: loss 0.6869907428820928 | Acc 61.63265306122449
Batch 13: loss 0.6864107847213745 | Acc 69.38775510204081
Batch 14: loss 0.6862963523183551 | Acc 58.77551020408164
Batch 15: loss 0.6861640095710755 | Acc 60.0
Batch 16: loss 0.6858026422560215 | Acc 64.08163265306122
Batch 17: loss 0.68585064831902 | Acc 57.14285714285714
Batch 18: loss 0.6858793861336179 | Acc 

In [39]:
btrainer.run_eval(btrainer.best_model, btrainer.val_loader)

(72.12827988338194, 0.5262049313442498, 0.6789789761815752)

In [44]:
btrainer.run_eval(btrainer.best_model, btrainer.test_loader)

(70.60228969636637, 0.5078939691060346, 0.6737378544923736)

In [41]:
torch.save(btrainer.best_model.cls.state_dict(), 'relnet_weights-16.pkl')

### Prototypical Network

In [ ]:
import torch
from models.attention.model import LabelImageAttention, LabelImagePrototypeModel
from models.embedding.model import ImageTextEmbedding, TextEncoder, ImageEncoder

from utils.prototype import class_prototype_inf
from models.metaclassifier.base import euclidean_distance
from models.metaclassifier.trainer import ControlledMetaTrainer
from models.metaclassifier.baselines import ProtoNet

encoder = torch.load('models/embedding/model/imgtext_model_trained.pth')
encoder.text_model.device = device
base_model = ProtoNet(encoder, class_prototype_inf, euclidean_distance, trainable_base=False)
btrainer = ControlledMetaTrainer(base_model, NUM_SHOTS, NUM_WAYS, dataset_config, device=device)

In [ ]:
btrainer.run_eval(btrainer.model, btrainer.val_loader)

In [ ]:
btrainer.run_eval(btrainer.best_model, btrainer.val_loader)